In [10]:
import pandas as pd

# Load the Excel file
file_path = "Candidate_Task_DailyReports.xlsx"   # keep file in same folder as script
df = pd.read_excel(file_path)

# Convert Date column to datetime and sort
df['Date'] = pd.to_datetime(df['Date'])
df = df.sort_values(by=['Employee', 'Task', 'Date'])

# Filter only tasks marked as "Not Done"
missed = df[df['Status'] == 'Not Done']

output = []

# Group by Employee & Task
for (emp, task), group in missed.groupby(['Employee', 'Task']):
    group = group.sort_values('Date')
    dates = group['Date'].dt.strftime('%Y-%m-%d').tolist()

    # Detect consecutive missed days
    consecutive = []
    temp = [dates[0]]

    for i in range(1, len(dates)):
        prev = pd.to_datetime(dates[i-1])
        curr = pd.to_datetime(dates[i])
        if (curr - prev).days == 1:  # consecutive
            temp.append(dates[i])
        else:
            consecutive.append(temp)
            temp = [dates[i]]
    consecutive.append(temp)

    # Add results
    for seq in consecutive:
        if len(seq) == 1:
            action = "Follow-up"
        else:
            action = "Escalate"
        output.append([emp, task, ", ".join(seq), action])

# Create final output table
result = pd.DataFrame(output, columns=['Employee', 'Task', 'Date(s) Missed', 'Action'])

# Save results to Excel
result.to_excel("Output_DailyReportReview_By_Shahnawaz.xlsx", index=False)

print("✅ Processing complete. Check 'Output_DailyReportReview.xlsx' for results.")

✅ Processing complete. Check 'Output_DailyReportReview.xlsx' for results.


In [11]:
output_df = pd.read_excel("Output_DailyReportReview_By_Shahnawaz.xlsx")
output_df.head(10)

,Employee,Task,Date(s) Missed,Action
0,Alice,Send Daily Sales Report,"2025-07-02, 2025-07-03",Escalate
1,Bob,Resolve High Priority Ticket,"2025-07-01, 2025-07-02",Escalate
